**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 4.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N967ccaecb71d45788c3a5889a5186fc7 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [5]:
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
ns = Namespace("http://somewhere#")

for s, p, o in g.triples((None,RDFS.subClassOf,ns.Person)):
  print(s)

from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT 
    ?Subject 
  WHERE { 
    ?Subject rdfs:subClassOf ns:Person 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)


for r in g.query(q1):
  print(r)

http://somewhere#Researcher
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [8]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s) 

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    print(s1)

q2 = prepareQuery('''
  SELECT ?Person
  WHERE { 
    {
    ?Person rdf:type ns:Person.
    } UNION {
      ?s1 rdfs:subClassOf ns:Person.
      ?Person rdf:type ?s1
    } 
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q2):
  print(r)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [12]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s1,p1,o1 in g.triples((s, None, None)):
      print(s1,p1,o1)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    for s2,p2,o2 in g.triples((s1, None, None)):
      print(s2,p2,o2)

q3 = prepareQuery('''
 SELECT ?Person ?Prop ?c
  WHERE {
   {
    ?Person rdf:type ns:Person.
    ?Person ?Prop ?o1

  }UNION{ 
     ?Person rdf:type ?s1.
     ?Person ?Prop ?o1.
     ?s1 rdfs:subClassOf ns:Person
   }
  }
  ''',
  initNs = { "ns": ns, "rdf": RDF, "rdfs": RDFS}
)

for s1 in g.query(q3):
  print(s1)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/2